# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [27]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [4]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
id            26248 non-null int64
categories    26248 non-null object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = messages.merge(categories, how='inner')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [7]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [8]:
# select the first row of the categories dataframe
row = categories.iloc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = list(row.apply(lambda x: x[:-2]))
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [9]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [10]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].astype(str).str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [11]:
# drop the original categories column from `df`
df.drop(columns='categories', inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [12]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [16]:
# check number of duplicates
len(df[df.duplicated()])

170

In [17]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [18]:
# check number of duplicates
len(df[df.duplicated()])

0

In [19]:
df.head(10)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26216 entries, 0 to 26385
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [21]:
df['original'][0]

'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'

In [24]:
for i, row in df.iterrows():
    if row['original']!='nan':
        print(row['original'], '\n')

Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi 

Cyclone nan fini osinon li pa fini 

Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi. 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately. 

facade ouest d Haiti et le reste du pays aujourd hui et ce soir 

Informtion au nivaux palais nationl 

Cyclone Coeur sacr de jesus 

Tanpri nou bezwen tant avek dlo nou zon silo mesi. 

Mwen ta renmen jouin messag yo. Merci 

Nou kwadebouke, nou gen pwoblem sant m yo nan santo 15 

Bon repo pa gen anyen menm grangou swaf 

M nan pv mvle plis enfomasyon sou 4636 

Mwen thomassin 32 nan pyron mwen ta renmen jwen yon ti dlo gras a dieu bo lakay mwen anfom se sel dlo nou bezwen 

Ann fel ansanm bezwen manje nan delma 75 nan riyel didine 

gen plis enfomasyon sou 4636 la pom w sim ap ka patisipe. 

Ko


Nou steel pa ka jwenn manje petion ville nan bte, nap mouri wi. 

Mwen egzateman akay nan yon zon yo rele toman km 43 vil ki tou prm nan se kabari nan yon katye yo rele kafou lachomy 

A kile ambassade americain ap ouve demain? 

Ns sommes une firme de construction S. A Nous recrutons 4 ingnieurs civils, 2 architectes et 2 nouveaux actionnaires pr repondre aux besoins du moment. 

Ki sakaft pou jakmel nou gen anpil moun nan kolj latrinite ki anba dekomb yo 

Eske moun ka rantre lakay yo? Kila pa la ka jwenn kouran? 

Et aussi on a besoin de l aide de toute sorte du cote de BON REPOS route ONA VILE ou les gens veulent vraiment qu on les aident 

NAN KI BLOK RI MUGUET LA YE SVP ESKE SE LAVIL OU PETIONVIL 

Please, call or write us if you need more information or if you need our help like beeing translators, food distributors or any services else from us. Th 

est-ce vrai?la ville politique du pays sera hinche 

NAN KI DOMAN WAP BEZWENM MWEN PALE KREYL, FRANS, MWATYE ANGL, MW 

A fmasyon

ADRES LA SE NEW LIFE BO KAY ST FLE 

NET ONE AK LATORTUE NT AP FONKSYONE NAN ZON DEKAWO, FREDO NET AP F 

Map mande ak leta peyi a et ak tout enstans ki konsn pou yo w koman yota fe pou yo wete pap mol sen nikola a nan sal ye laa paske moune kap vann yo ar 

iv DOUBL PRI TOUT MACHANDISE MEM YON GOD MAYI AP VANN 45G SE MR EROLD DEPUIS MOLE-SAINT-NICOLAS 

Nou pa gen cyber nan zone ki en fonktyon. 

ures paske toutes les marchandises st montes tet ng, ti mamit diri 40g. 

bonjou mwen an danje kat mwen fini kijan nou ka edem. 

A J B D M Association jeune bon repos pour le developement mole st nicolas nou mande pou yo pote manje pou moun mol sen nikola yo mamit diri vann 5o dola 

moun yo paka jwenn manje lafin ap touye yo voye lapolis pou fer machand yo desann manje yo sou mache a 

l y a un probleme de signal qui empchent transactions 

Bonjou eske nou ka dim ki kote mwen ka jwenn yon biwo western union ki ouv nan kapital la? 

Nou nan lopital imakile konsepsyon okay. Adres intnet la se

Mwen petion vil mwen bezwen rele fanmiy mwen men mwen pagen kat. mete yon kat poumwen svp mesi. 

Nan zile la tti nou gen plizy moun ki mourinan kapital la men tout rs ki pa mouri yo oblije kite kapital la pou vini moun sa yo a 2men vid yo bezwen  

Nou nan lespinas zon fjak prolonjet kay nou kraze ak tout sa nou te posede nou pedi fanmi ak zanmi, nou bezwen manje. Mesi 

Mwen rete bit bwaye kwade misyon tout popilasyon sa depi la katatrof sa pase a pa janm gen anyen ki vin distribiye nan zon sa m ta renmen otorite konsne 

yo fe yon fason pou zon sa paske tout moun ap mouri grangou mesi 

Bonjou ! Mwen an haiti : bezwen pa m ak paran m yo se jwenn manje pou nou manje ak kote pou domi. Adres kote m rete se bore pot lathan ryl m b c nimewo 

Nou resevwa anpil moun ki grav nan ptpe men nou manke dokt spesyalis nou bezwen ed nan lopital imakile ak lapwent mesi 

mwen selman genyen rad kite sou mwen jounen madi a silvoupl ede m aktyelman mwen trouve m jis Pdep 

Mwen viktim mwen bezwen asi

Mwen nan Bonbadopolis Ayiti. Anpil nan moun ki pa mouri nan katastw 

Svp pandan tranbleman de terre a te pase a li kraze kay nou te rete a epi genyen blok ki tonbe nou silvoupl nou bezwen yon aide se elv lekol nou ye nou 

Mwen nan zonn Jan-Deni pwemye sektyon petit rivy de latibonit. 

Mou nan depatman lwes zone gran gwave localite foche nou grangou nou nou nan la ri mwen se agar larose. 

Gen yon bann viktim ki rive nan lopital Kl Erez Machan Desalin.Dapri paran viktim yo, gen anpil volonte nan mitan medsen ayisyen ki te deja sou plas men 

she is really sick she need your help. please use my phone number to get more informations about her. We waiting for your answers. 

Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi 

Je tiens  vous remercier d avoir choisir d aider nous les haitiens en Haiti. Je profite de cette meme occasion pour vous saluer dans le nom du seigneur 

Se leyo

...

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

nan 

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [26]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('MessageCategory', engine, index=False)

In [28]:
# connect to the database
conn = sqlite3.connect('DisasterResponse.db')

# run a query
df_sql=pd.read_sql('SELECT * FROM MessageCategory', conn)
df_sql.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.